# Image Classification

In [1]:
# Import Dependencies

# General libraries / functions
import pandas as pd
import numpy as np
import pickle

# Classification libaries / functions
from mlxtend.classifier import EnsembleClassifier
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.lda import LDA
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import BernoulliRBM
from sklearn.pipeline import Pipeline
from sklearn.qda import QDA
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier

# Display libraries / functions
import pprint
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Pickling functions
def pickle_it(data, filename, python_version=3):
    """
    In:
    data = the data you want to pickle (save)
    filename = file name where you want to save the data
    python_version = the python version where you will be opening the pickle file
    
    Out:
    Saves a pickle file with your data to to the filename you specify
    """
    with open(filename, "wb") as picklefile:
        pickle.dump(data, picklefile, protocol=python_version)

def load_pickle(filename):
    """
    In:
    filename = name of the pickle file you want to open (e.g "my_pickle.pkl")
    
    Out:
    Opens and returns the content of the picklefile to a variable of your choice
    """
    with open(filename, "rb") as picklefile: 
        return pickle.load(picklefile)

In [3]:
# Load training data
training_processed_zipped = load_pickle("training_processed_zipped.pkl")
train_images = training_processed_zipped["images"]
train_signs = training_processed_zipped["signs"]

# Load testing data
test_processed_zipped = load_pickle("test_processed_zipped.pkl")
test_images = test_processed_zipped["images"]
test_signs = test_processed_zipped["signs"]

In [4]:
# Vectorize images (flatten to single row of pixels)
if len(train_images) > 1:
    train_images = pd.DataFrame([np.hstack((a.ravel(), b.ravel())) \
                                 for a, b in zip(train_images[0], train_images[1])]).fillna(0)
    test_images = pd.DataFrame([np.hstack((a.ravel(), b.ravel())) \
                                for a, b in zip(test_images[0], test_images[1])]).fillna(0)
else:
    train_images = pd.DataFrame([img.ravel() for img in train_images[0]]).fillna(0)
    test_images = pd.DataFrame([img.ravel() for img in test_images[0]]).fillna(0)

In [5]:
# Use Principal Components Analysis (PCA) to reduce feature space / dimensionality
pca = PCA(n_components=188)  # Optimal n (as determined by grid search)
pca.fit(train_images)

train_images = pca.transform(train_images)
test_images = pca.transform(test_images)

In [ ]:
"""
ALTERNATE FEATURE / DIMENSION REDUCTION APPROACH (RESTRICTED BOLTZMAN MACHINE):
(NOT USED IN FINAL MODEL)

rbm = BernoulliRBM(n_components=100, learning_rate=0.1, batch_size=10, n_iter=10, verbose=1)
rbm.fit(train_images)

train_images = rbm.transform(train_images)
test_images = rbm.transform(test_images)
"""

In [26]:
# All individual models tried
models = {
        "linear_SVM": LinearSVC(penalty="l1", C=0.61, dual=False),
        "SVM": SVC(),
        "RBF SVM": SVC(gamma=2, C=1),
        "KNN4": KNeighborsClassifier(4),
        "decision_tree": DecisionTreeClassifier(min_samples_leaf=32),
        "random_forest": RandomForestClassifier(n_estimators=80),
        # "gaussianNB": GaussianNB(),
        # "bernoulliNB": BernoulliNB(),
        # "multinomialNB": MultinomialNB(),
        "logistic": LogisticRegression("l1", C=0.16),
        "LDA": LDA(),
        "QDA": QDA(),
        "SGD_hinge": SGDClassifier(loss="hinge", penalty="l1"),
        "SGD_huber": SGDClassifier(loss="modified_huber", penalty="l1"),
        "SGD_log": SGDClassifier(loss="log", penalty="l1"),
        "SGD_squared_hinge": SGDClassifier(loss="squared_hinge", penalty="l1"),
        "SGD_perceptron": SGDClassifier(loss="perceptron", penalty="l1"),
        "adaboost": AdaBoostClassifier(n_estimators=200),
        "gradient_boosting": GradientBoostingClassifier()
    }

In [27]:
for name, model in models.items():
    model.fit(train_images, train_signs)
    print(name)
    print("Accuracy: %s" % accuracy_score(test_signs, model.predict(test_images)))
    print(classification_report(test_signs, model.predict(test_images)))
    print("\n")

adaboost
Accuracy: 0.257966616085
             precision    recall  f1-score   support

          A       0.21      0.16      0.18        97
          B       0.42      0.30      0.35       102
          C       0.32      0.17      0.22       112
       Five       0.32      0.09      0.14       134
      Point       0.23      0.67      0.35       119
          V       0.18      0.13      0.15        95

avg / total       0.28      0.26      0.23       659



random_forest
Accuracy: 0.283763277693
             precision    recall  f1-score   support

          A       0.25      0.30      0.27        97
          B       0.43      0.21      0.28       102
          C       0.48      0.26      0.34       112
       Five       0.24      0.13      0.17       134
      Point       0.24      0.70      0.36       119
          V       0.42      0.08      0.14        95

avg / total       0.34      0.28      0.26       659



SGD_squared_hinge
Accuracy: 0.376327769347
             precision    

/Users/seanosier/anaconda/lib/python3.4/site-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/seanosier/anaconda/lib/python3.4/site-packages/sklearn/qda.py:133: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")





QDA
Accuracy: 0.215477996965
             precision    recall  f1-score   support

          A       0.34      0.12      0.18        97
          B       1.00      0.04      0.08       102
          C       0.90      0.08      0.15       112
       Five       0.00      0.00      0.00       134
      Point       0.19      0.98      0.32       119
          V       0.00      0.00      0.00        95

avg / total       0.39      0.22      0.12       659



LDA
Accuracy: 0.396054628225
             precision    recall  f1-score   support

          A       0.58      0.26      0.36        97
          B       0.32      0.74      0.45       102
          C       0.79      0.54      0.65       112
       Five       0.32      0.19      0.23       134
      Point       0.32      0.39      0.35       119
          V       0.35      0.29      0.32        95

avg / total       0.44      0.40      0.39       659



SGD_perceptron
Accuracy: 0.361153262519
             precision    recall  f1-scor

/Users/seanosier/anaconda/lib/python3.4/site-packages/sklearn/lda.py:371: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


In [39]:
# Final models selected to be included in ensemble model
# Models to include can vary based on the input images, so should evaulate models to include in the ensemble each time
ensemble_models = {
        "LDA": LDA(),
        "SVM": SVC()
    }

In [40]:
master_model = EnsembleClassifier(ensemble_models.values())
master_model.fit(train_images, train_signs)
print("Ensemble")
print("Accuracy: %s" % accuracy_score(test_signs, master_model.predict(test_images)))
print(classification_report(test_signs, master_model.predict(test_images)))
print("\n")

Ensemble
Accuracy: 0.412746585736
             precision    recall  f1-score   support

          A       0.54      0.34      0.42        97
          B       0.32      0.80      0.46       102
          C       0.79      0.55      0.65       112
       Five       0.39      0.20      0.27       134
      Point       0.34      0.42      0.38       119
          V       0.36      0.19      0.25        95

avg / total       0.46      0.41      0.40       659





/Users/seanosier/anaconda/lib/python3.4/site-packages/sklearn/lda.py:371: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


### Results

**Current Best:**  
*Ensemble SVM + LDA*  
**41.3%** Accuracy

**History:**
1. Baseline: Dumb = 16.6%
2. Best 1: Multinomial Naive Bayes = ~30%
 - *Initial image processing and models*
3. Best 2: Logistic Regression = 32.6%
 - *More image processing*
4. Best 3: Logistic Regression = 33.8%
 - *Inverted equalized hist color*
5. Best 4: Ensemble Logistic + Linear SVM = 33.9%
6. Best 5: Ensemble Logistic + Linear SVM = 35.5%
 - *Raw grayscale PCA = 100*
7. Best 6: Ensemble Logistic + Linear SVM = 37.9%
 - *Optimized PCA = 188*
8. Best 7: Ensemble Logistic + Linear SVM = 38.4%
 - *Optimized Logistic, C = 0.16*
9. Best 8: Ensemble Logistic + Linear SVM = 38.5%
 - *Optimized Linear SVM, C = 0.61*
10. Best 9: Ensemble Logistic + Linear SVM = 38.8%
 - *Inverted equalized hist color with circle*
11. Best 10: Ensemble SVM + LDA = 41.3%
 - *Refined models included in ensemble model*

### Opportunities for Additional Improvement
- Image processing to better isolate the hand from background?
- Feed multiple image types into the models (e.g. a grayscale image __*and*__ a skeletonized one)?
- Refine ensemble model further by re-optimizing all included models for the latest image inputs?
- Use voting/probability weights in the ensemble?
- Neural networks (specifically convolutional)?